In [2]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp

project_name = 'apache-isis'
rootdir = 'raw_depends/' + project_name
json_name_dict = {}
json_result_dict = {}
for root, dirs, files in os.walk(rootdir):
   
    #print(files)
    for element in files:
        
        with open(rootdir + '/' + element) as f:
            tmp = json.load(f)
        
        element = element.split('_')
        json_name_dict[element[0]] = element[1]
        json_result_dict[element[0]] = tmp

In [4]:
json_name_dict

{'0': 'apache-isis-1.10.0.json',
 '10': 'apache-isis-1.15.0.json',
 '11': 'apache-isis-1.15.1.json',
 '12': 'apache-isis-1.16.0.json',
 '13': 'apache-isis-1.16.1.json',
 '14': 'apache-isis-1.16.2.json',
 '15': 'apache-isis-1.17.0.json',
 '16': 'apache-isis-1.6.0.json',
 '17': 'apache-isis-1.7.0.json',
 '18': 'apache-isis-1.8.0.json',
 '19': 'apache-isis-1.9.0.json',
 '1': 'apache-isis-1.11.0.json',
 '20': 'apache-isis-2.0.0.json',
 '2': 'apache-isis-1.11.1.json',
 '3': 'apache-isis-1.12.0.json',
 '4': 'apache-isis-1.12.1.json',
 '5': 'apache-isis-1.12.2.json',
 '6': 'apache-isis-1.13.0.json',
 '7': 'apache-isis-1.13.1.json',
 '8': 'apache-isis-1.13.2.json',
 '9': 'apache-isis-1.14.0.json'}

In [4]:
def common_member(a,b):
    a_set = set(a)
    b_set = set(b)
    if len(a_set.intersection(b_set)) > 0:
        return(a_set.intersection(b_set))
    else:
        return set()

def compare_similar(right, left):
    difference = list();
    for root, dirs, files in os.walk(right):
        for name in files:
            path = os.path.join(root, name);
            # check if file is same
            if fileIsSame(right, left, path):
                
                if os.path.isdir(path):
                    # recursively check subdirs
                    difference.extend(compare_similar(path, left));
                difference.append(path)
                
            #else:
                # count file as difference
                #difference.append(path);

    return difference;

def fileIsSame(right, left, path):
    return os.path.exists (os.path.join(left, path.replace(right, '')));

ground_truth_dict = {}
max_len_tmp = 0
root_dir = 'raw_sourcecode/' + project_name + '/'
for i in range(10,20):
    file1 = str(root_dir +json_name_dict[str(i)]).replace('.json', '') + '/'
    for j in range(1,11):
        file2 = str(root_dir + json_name_dict[str(i-j)]).replace('.json', '') + '/'
        
        #test = compare_similar()
        #print(file1, file2)
        if (j == 1):
            prev_tmp = compare_similar(file1, file2)
        else:
            tmp = compare_similar(file1, file2)
            tmp = common_member(prev_tmp, tmp)
            #print(len(tmp))
            prev_tmp = tmp
            
    if len(prev_tmp) > max_len_tmp:
        max_len_tmp = len(prev_tmp)
    #print(len(prev_tmp))
    ground_truth_dict[str(i)] = prev_tmp

In [10]:
len(ground_truth_dict)

10

In [16]:
for i in range(10):
    print(json_name_dict[str(i+10)].split('.json')[0])

apache-isis-1.15.0
apache-isis-1.15.1
apache-isis-1.16.0
apache-isis-1.16.1
apache-isis-1.16.2
apache-isis-1.17.0
apache-isis-1.6.0
apache-isis-1.7.0
apache-isis-1.8.0
apache-isis-1.9.0


In [14]:
for i in range(len(json_name_dict)):
    print(json_name_dict[str(i)].split('.json')[0])

apache-isis-1.10.0
apache-isis-1.11.0
apache-isis-1.11.1
apache-isis-1.12.0
apache-isis-1.12.1
apache-isis-1.12.2
apache-isis-1.13.0
apache-isis-1.13.1
apache-isis-1.13.2
apache-isis-1.14.0
apache-isis-1.15.0
apache-isis-1.15.1
apache-isis-1.16.0
apache-isis-1.16.1
apache-isis-1.16.2
apache-isis-1.17.0
apache-isis-1.6.0
apache-isis-1.7.0
apache-isis-1.8.0
apache-isis-1.9.0
apache-isis-2.0.0


In [24]:
depth=2
rootdir_arr = []
rootdir = 'raw_sourcecode'
for root, dirs, files in os.walk(rootdir):
    if root.count(os.sep) == depth and project_name in str(root):
        #print(root)
        #dir_arr.append(root)
        rootdir_arr.append(root)

In [25]:
rootdir_arr

['raw_sourcecode\\apache-isis\\apache-isis-1.10.0',
 'raw_sourcecode\\apache-isis\\apache-isis-1.11.0',
 'raw_sourcecode\\apache-isis\\apache-isis-1.11.1',
 'raw_sourcecode\\apache-isis\\apache-isis-1.12.0',
 'raw_sourcecode\\apache-isis\\apache-isis-1.12.1',
 'raw_sourcecode\\apache-isis\\apache-isis-1.12.2',
 'raw_sourcecode\\apache-isis\\apache-isis-1.13.0',
 'raw_sourcecode\\apache-isis\\apache-isis-1.13.1',
 'raw_sourcecode\\apache-isis\\apache-isis-1.13.2',
 'raw_sourcecode\\apache-isis\\apache-isis-1.14.0',
 'raw_sourcecode\\apache-isis\\apache-isis-1.15.0',
 'raw_sourcecode\\apache-isis\\apache-isis-1.15.1',
 'raw_sourcecode\\apache-isis\\apache-isis-1.16.0',
 'raw_sourcecode\\apache-isis\\apache-isis-1.16.1',
 'raw_sourcecode\\apache-isis\\apache-isis-1.16.2',
 'raw_sourcecode\\apache-isis\\apache-isis-1.17.0',
 'raw_sourcecode\\apache-isis\\apache-isis-1.6.0',
 'raw_sourcecode\\apache-isis\\apache-isis-1.7.0',
 'raw_sourcecode\\apache-isis\\apache-isis-1.8.0',
 'raw_sourcecod

In [5]:
root_dir = 'raw_sourcecode/' + project_name + '/'
for i in range(10,20):
    file1 = str(root_dir +json_name_dict[str(i)]).replace('.json', '') + '/'
    for j in range(1,11):
        file2 = str(root_dir + json_name_dict[str(i-j)]).replace('.json', '') + '/'
        
        #test = compare_similar()
        #print(file1, file2)
        if (j == 1):
            prev_tmp = compare_similar(file1, file2)
        else:
            tmp = compare_similar(file1, file2)
            tmp = common_member(prev_tmp, tmp)
            #print(len(tmp))
            prev_tmp = tmp
            
    if len(prev_tmp) > max_len_tmp:
        max_len_tmp = len(prev_tmp)
    #print(len(prev_tmp))
    ground_truth_dict[str(i)] = prev_tmp

In [6]:
max_len_tmp

3504

In [18]:
    
f = open('project_list.txt', 'a')
project_name = 'rzwitserloot-lombok'
to_write = project_name + '\n'
f.write(to_write)
f.close()

In [19]:
df = pd.read_csv('Final_Results/apache-cassandra/apache-cassandra-apache-cassandra-3.2.1_class.csv')

In [20]:
df.head()

,file,class,type,cbo,wmc,dit,rfc,lcom,totalMethods,staticMethods,...,numbersQty,assignmentsQty,mathOperationsQty,variablesQty,maxNestedBlocks,anonymousClassesQty,subClassesQty,lambdasQty,uniqueWordsQty,modifiers
0,C:\Users\tanji\Desktop\FIT4003\raw_sourcecode\...,WordCount,class,21,16,6,32,1,2,1,...,15,22,3,16,2,0,4,0,118,1
1,C:\Users\tanji\Desktop\FIT4003\raw_sourcecode\...,WordCountCounters,class,18,7,5,23,1,2,1,...,6,8,1,7,1,0,3,0,91,1
2,C:\Users\tanji\Desktop\FIT4003\raw_sourcecode\...,WordCountSetup,class,13,15,1,28,15,7,7,...,9,19,10,18,2,0,0,0,110,1
3,C:\Users\tanji\Desktop\FIT4003\raw_sourcecode\...,org.apache.cassandra.triggers.AuditTrigger,class,12,3,1,17,0,2,1,...,0,6,0,6,1,0,0,0,21,1
4,C:\Users\tanji\Desktop\FIT4003\raw_sourcecode\...,org.apache.cassandra.thrift.AuthenticationExce...,class,17,60,4,32,267,30,3,...,7,25,2,17,2,0,4,0,136,1


In [21]:
test = df.sum()

In [22]:
test

file                    C:\Users\tanji\Desktop\FIT4003\raw_sourcecode\...
class                   WordCountWordCountCountersWordCountSetuporg.ap...
type                    classclassclassclassclassclassclassclassclassc...
cbo                                                                 19845
wmc                                                                 36640
dit                                                                  3896
rfc                                                                 39482
lcom                                                               179293
totalMethods                                                        17786
staticMethods                                                        3506
publicMethods                                                       13747
privateMethods                                                       2422
protectedMethods                                                      666
defaultMethods                        

In [24]:
df.columns

Index(['file', 'class', 'type', 'cbo', 'wmc', 'dit', 'rfc', 'lcom',
       'totalMethods', 'staticMethods', 'publicMethods', 'privateMethods',
       'protectedMethods', 'defaultMethods', 'abstractMethods', 'finalMethods',
       'synchronizedMethods', 'totalFields', 'staticFields', 'publicFields',
       'privateFields', 'protectedFields', 'defaultFields', 'finalFields',
       'synchronizedFields', 'nosi', 'loc', 'returnQty', 'loopQty',
       'comparisonsQty', 'tryCatchQty', 'parenthesizedExpsQty',
       'stringLiteralsQty', 'numbersQty', 'assignmentsQty',
       'mathOperationsQty', 'variablesQty', 'maxNestedBlocks',
       'anonymousClassesQty', 'subClassesQty', 'lambdasQty', 'uniqueWordsQty',
       'modifiers'],
      dtype='object')

In [25]:
test['modifiers']

174130